In [1]:
# Librerías necesarias

from datetime import datetime, date
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import (
    col, max, count, countDistinct, first, regexp_replace, lower, coalesce,
    year, month, to_date, lit, row_number, min, trim
)
from pyspark.sql.types import IntegerType, DateType, StructType, StringType, LongType, DoubleType, StructField
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel
import pyspark.sql.functions as f

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1714242990235_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Se inicializa una sesión de spark
spark = SparkSession.builder \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Se declara el path de donde se leerán los parquets
path_data = "s3://proyectoprofeco/data/parquets"
df_profecoBasicos = spark.read.parquet(f"{path_data}/catalogo=basicos")

# Ruta al s3 que contiene el archivo de marcas únicas
ruta_marcas = "s3://proyectoprofeco/data/raw/marcas_basicos.csv"

# Se carga un catalogo de marcas únicas
catalogo_df = spark.read.csv(ruta_marcas, header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_profecoBasicos.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+------+-------------+---------------+--------------------+--------------------+--------------------+----------------+--------------+---------+----------+----+
|            producto|        presentacion|               marca|           categoria|precio|fecharegistro|cadenacomercial|                giro|     nombrecomercial|           direccion|          estado|     municipio|  latitud|  longitud|anio|
+--------------------+--------------------+--------------------+--------------------+------+-------------+---------------+--------------------+--------------------+--------------------+----------------+--------------+---------+----------+----+
|pollo rostizado. ...|pieza charola o b...|                  sm|alimentos cocinad...|  84.0|   2018-09-26|   mega soriana|supermercado / ti...|mega soriana sucu...|calz. mexico-tacu...|ciudad_de_mexico|miguel hidalgo|19.460236|-99.193576|2018|
|     polvo p/hornear|  

In [10]:
# Se homologan las marcas dejando marcas unicas
df_profecoBasicos = (df_profecoBasicos
    .withColumn("norm_marca", lower(regexp_replace("marca", "[\- ]", "")))
    .join(catalogo_df, col("norm_marca") == catalogo_df.marca, "left")
    .withColumnRenamed("marca_distinct", "new_norm_marca")
    .withColumn("norm_marca", coalesce(col("new_norm_marca"), col("norm_marca")))
    .drop("new_norm_marca", "marca")
    .withColumnRenamed("norm_marca", "marca")
)

df_profecoBasicos.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- fecharegistro: date (nullable = true)
 |-- cadenacomercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombrecomercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)
 |-- anio: integer (nullable = true)
 |-- marca: string (nullable = true)

In [11]:
# Se guarda la información particionada en parquets
(df_profecoBasicos
     .write.parquet(
         "s3://proyectoprofeco/data/basicos/", 
         mode="overwrite", 
         partitionBy=["estado", "anio"]
     )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…